In [ ]:
from binance.spot import Spot as Client
import pandas as pd
import numpy as np
import dotenv
import matplotlib.pyplot as plt
import os
from aperture import OptionMarkPriceClient

In [ ]:
dotenv.load_dotenv()

In [ ]:
client = Client(
    os.getenv("BINANCE_API_KEY"),
    os.getenv("BINANCE_API_SECRET"), 
)

In [ ]:
client = OptionMarkPriceClient(
    api_key=os.getenv("BINANCE_API_KEY"),
    secret_key=os.getenv("BINANCE_API_SECRET"),
)

call_chain_df = await client.get_option_chain_df(date='2023-07-07', coin='ETH', option_type='CALL')
put_chain_df = await client.get_option_chain_df(date='2023-07-07', coin='ETH', option_type='PUT')

In [ ]:
put_chain_df.sort_values('strike', ascending=False, inplace=True)
call_chain_df.sort_values('strike', ascending=True, inplace=True)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(put_chain_df['strike'], put_chain_df['markIV'], label='put mark IV', linestyle='--', color='red')
plt.plot(call_chain_df['strike'], call_chain_df['markIV'], label='put mark IV', linestyle='--', color='green')
plt.scatter(put_chain_df['strike'], put_chain_df['askIV'], label='put ask IV', marker='v', color='red')
plt.scatter(put_chain_df['strike'], put_chain_df['bidIV'], label='put bid IV', marker='^', color='red')
plt.scatter(call_chain_df['strike'], call_chain_df['askIV'], label='call ask IV', marker='v', color='green', alpha=0.5)
plt.scatter(call_chain_df['strike'], call_chain_df['bidIV'], label='call bid IV', marker='^', color='green', alpha=0.5)
plt.legend()
plt.grid()
# plt.plot(call_chain_df['strike'], put_chain_df['markIV'], label='put mark IV')

In [ ]:
put_chain_df

In [ ]:
import math
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """
    This function returns the Black-Scholes call price.
    """
    d1 = (math.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

def vega(S, K, T, r, sigma):
    """
    This function returns the Vega of the option, which is needed in the calculation of implied volatility.
    """
    d1 = (math.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * math.sqrt(T))
    return S * norm.pdf(d1) * math.sqrt(T)

def implied_vol_newton_raphson(S, K, T, r, market_price, sigma_init=0.5, max_iter=100, tol=1e-5):
    """
    This function returns the implied volatility using Newton-Raphson method.
    """
    sigma = sigma_init
    for i in range(max_iter):
        price = black_scholes_call(S, K, T, r, sigma)
        v = vega(S, K, T, r, sigma)
        price_diff = price - market_price  # f(sigma)

        # if the difference is very small, break out the loop
        if abs(price_diff) < tol:
            return sigma

        # Newton-Raphson formula
        sigma = sigma - price_diff/v  # sigma - f(sigma)/f'(sigma)

    # If volatility is not found after max iterations, raise an exception
    raise Exception('Implied volatility not found')

# usage example:
S = 100  # Underlying asset price
K = 100  # Strike price
T = 1    # Time to maturity
r = 0.05 # Risk-free interest rate
market_price = 10  # Market price of the option

implied_vol = implied_vol_newton_raphson(S, K, T, r, market_price)
print('Implied Volatility:', implied_vol)


In [ ]:
(pd.Timestamp('2023-07-07 08').value / 1_000_000_000  - time.time()) / (3600 * 24 * 365)

In [ ]:
call_chain_df

In [ ]:
implied_vol_newton_raphson(
    1868.1, 
    1400, 
    (pd.Timestamp('2023-07-07 08').value / 1_000_000_000  - time.time()) / (3600 * 24 * 365), 
    0, 
    445.6,
    max_iter=100
)

In [ ]:
put_chain_df